# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 11:24AM,263482,10,ALK38257,Alikai Health,Released
1,May 23 2023 11:24AM,263482,10,ALK38260,Alikai Health,Released
2,May 23 2023 11:24AM,263482,10,ALK38649,Alikai Health,Released
3,May 23 2023 11:24AM,263482,10,ALK38689,Alikai Health,Released
4,May 23 2023 11:24AM,263482,10,ALK38691,Alikai Health,Released
5,May 23 2023 11:24AM,263482,10,ALK38707,Alikai Health,Released
6,May 23 2023 11:24AM,263482,10,ALK38709,Alikai Health,Released
7,May 23 2023 11:23AM,263474,15,30016738,"Alliance Pharma, Inc.",Released
8,May 23 2023 11:23AM,263474,15,30016740,"Alliance Pharma, Inc.",Released
9,May 23 2023 11:23AM,263474,15,30016741,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,263477,Released,1
29,263479,Released,2
30,263480,Executing,1
31,263481,Released,5
32,263482,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263477,NaN,NaN,1.0
263479,NaN,NaN,2.0
263480,NaN,1.0,NaN
263481,NaN,NaN,5.0
263482,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0.0,2.0,1.0
263409,0.0,0.0,26.0
263421,0.0,0.0,7.0
263434,7.0,11.0,2.0
263439,0.0,19.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0,2,1
263409,0,0,26
263421,0,0,7
263434,7,11,2
263439,0,19,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,0,2,1
1,263409,0,0,26
2,263421,0,0,7
3,263434,7,11,2
4,263439,0,19,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,,2,1
1,263409,,,26
2,263421,,,7
3,263434,7,11,2
4,263439,,19,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 11:24AM,263482,10,Alikai Health
7,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc."
66,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc."
73,May 23 2023 11:12AM,263481,10,"Methapharm, Inc."
78,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc."
104,May 23 2023 11:09AM,263480,15,Lonza
105,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC
107,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC"
108,May 23 2023 10:58AM,263476,19,Else Nutrition
109,May 23 2023 10:55AM,263475,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2023 11:24AM,263482,10,Alikai Health,,,7
1,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc.",,,59
2,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc.",,,7
3,May 23 2023 11:12AM,263481,10,"Methapharm, Inc.",,,5
4,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc.",,,26
5,May 23 2023 11:09AM,263480,15,Lonza,,1,
6,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,,,2
7,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",,,1
8,May 23 2023 10:58AM,263476,19,Else Nutrition,,,1
9,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:24AM,263482,10,Alikai Health,7,,
1,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc.",59,,
2,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc.",7,,
3,May 23 2023 11:12AM,263481,10,"Methapharm, Inc.",5,,
4,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc.",26,,
5,May 23 2023 11:09AM,263480,15,Lonza,,1,
6,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,2,,
7,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",1,,
8,May 23 2023 10:58AM,263476,19,Else Nutrition,1,,
9,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:24AM,263482,10,Alikai Health,7,,
1,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc.",59,,
2,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc.",7,,
3,May 23 2023 11:12AM,263481,10,"Methapharm, Inc.",5,,
4,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc.",26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:24AM,263482,10,Alikai Health,7.0,NaN,NaN
1,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc.",59.0,NaN,NaN
2,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc.",7.0,NaN,NaN
3,May 23 2023 11:12AM,263481,10,"Methapharm, Inc.",5.0,NaN,NaN
4,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc.",26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:24AM,263482,10,Alikai Health,7.0,0.0,0.0
1,May 23 2023 11:23AM,263474,15,"Alliance Pharma, Inc.",59.0,0.0,0.0
2,May 23 2023 11:12AM,263421,15,"Alliance Pharma, Inc.",7.0,0.0,0.0
3,May 23 2023 11:12AM,263481,10,"Methapharm, Inc.",5.0,0.0,0.0
4,May 23 2023 11:10AM,263409,15,"Person & Covey, Inc.",26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2634657,32.0,1.0,0.0
15,1317086,93.0,3.0,0.0
19,2107653,44.0,56.0,3.0
20,526906,2.0,12.0,7.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2634657,32.0,1.0,0.0
1,15,1317086,93.0,3.0,0.0
2,19,2107653,44.0,56.0,3.0
3,20,526906,2.0,12.0,7.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,32.0,1.0,0.0
1,15,93.0,3.0,0.0
2,19,44.0,56.0,3.0
3,20,2.0,12.0,7.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,15,Released,93.0
2,19,Released,44.0
3,20,Released,2.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,3.0,7.0
Executing,1.0,3.0,56.0,12.0
Released,32.0,93.0,44.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,3.0,7.0
1,Executing,1.0,3.0,56.0,12.0
2,Released,32.0,93.0,44.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,3.0,7.0
1,Executing,1.0,3.0,56.0,12.0
2,Released,32.0,93.0,44.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()